# **Reward Calculation System For CRASHR ISPO**

### **1. Overview**


#### **1.1 Purpose**


This document explains the reward calculation system for delegators, highlighting how data is fetched, how rewards are calculated, and how bonuses are applied based on tiered rules

#### **1.2 Workflow Summary**


1. Fetch Delegator History(stake addresses, pool, stakes)

2. Calculate rewards using provided formulas

3. Fetch asset holdings (1/1 NFTs and specific policyID)

4. Calculate bonuses based on tiered rules

#### **1.3 Necessary Imports**

In [27]:
import requests
import pandas as pd

### **2. Fetching Delegator Data**

- This endpoint returns the delegator history of all stakes based on the epoch range and list of the pools that is included.

- This uses the Koios API under the hood

- This endpoint accepts four parameters:

    * **main_pool_start_epoch** - The epoch where we begin fetching the delegator history for the main pool only.

    * **all_pools_start_epoch** - The epoch where we start fetching the history for both the main pool and partner pools.

    * **end_epoch** - This the end of the epoch range.
    
    * **pool_ids** - The list of pools for which we fetch the delegator history, including both the main pool and the partner pools.

- This endpoint returns the delegator history and it also returns the total ada staked by all participants.

In [28]:
main_pool_start_epoch = 451 

all_pools_start_epoch = 459 

end_epoch = 502 

pool_ids = {
    "CRASH": "pool1j8zhlvakd29yup5xmxtyhrmeh24udqrgkwdp99d9tx356wpjarn",       # > Main Pool
    "HAPPY": "pool1a8n7f97dmgtgrnl53exccknjdchqxrr9508hlxlgqp6xvjmzhej",       #------------
    "KTOP": "pool135h773klt7djljmyawkh88e8qc457wxqfhc6j9h6y9n4y3sgh7t",        #------------->  PARTNER POOLS
    "ELEMENTAL": "pool19ut4284xy9p82dd0cglzxweddfqw73yennkjk6mmp650chnr6lz",   #------------->
    "RUMOR": "pool1c30lqt59t8sjn5lg04r5wk5eucxa5h9cj05xs9gzc8lngl4cmta"        #------------
}

In [29]:
# Request to the api

fetchParams = {
    "MainPoolStartEpoch": main_pool_start_epoch,
    
    "StartEpoch": all_pools_start_epoch, 
    
    "EndEpoch": end_epoch,
    
    "PoolIds": pool_ids
}

url = "http://localhost:5134/fetch-delegator-data" #endpoint
response = requests.post(url, json=fetchParams)

In [30]:
response_data = response.json()

total_ada = response_data.get('totalAda', 0)
delegator_data = response_data.get('result', [])


# Display the delegator history from the given epoch-range in a table
df = pd.DataFrame(delegator_data)
df.rename(columns={"amount": "amount(lovelace)"}, inplace=True)
display(df)

# Display total ADA staked by all participants
print(f"Total ADA: {total_ada}")


,pool_name,pool_id,epoch,stake_address,amount(lovelace)
0,CRASH,pool1j8zhlvakd29yup5xmxtyhrmeh24udqrgkwdp99d9t...,453,stake1u8x78suakz2juuwhatcvwvqrh8frsjegcusc0n75...,215567492003
1,CRASH,pool1j8zhlvakd29yup5xmxtyhrmeh24udqrgkwdp99d9t...,453,stake1uxdtlxetm6xqy8tgt949p04y6x2gzq3vx2w3c0wh...,163151351971
2,CRASH,pool1j8zhlvakd29yup5xmxtyhrmeh24udqrgkwdp99d9t...,453,stake1uxhjmz3lwc4tz99tf0y94xe3yluwrkes0lzjwp6w...,85239750965
3,CRASH,pool1j8zhlvakd29yup5xmxtyhrmeh24udqrgkwdp99d9t...,453,stake1uya822qae2zg62wmffheq02xenrzvv2yz34m3gcq...,84513682148
4,CRASH,pool1j8zhlvakd29yup5xmxtyhrmeh24udqrgkwdp99d9t...,453,stake1uxrrenlt5amge790qz48cq4rj8ah525kfcsntuzc...,58623899817
...,...,...,...,...,...
71394,RUMOR,pool1c30lqt59t8sjn5lg04r5wk5eucxa5h9cj05xs9gzc...,502,stake1u82f6gnv5thm2505gt99qwa3dh3c6d9dfvlszfjl...,1480995
71395,RUMOR,pool1c30lqt59t8sjn5lg04r5wk5eucxa5h9cj05xs9gzc...,502,stake1uxxv06v0jtrfqk3qjzlnphn39ntm99qa795z82mr...,1015524
71396,RUMOR,pool1c30lqt59t8sjn5lg04r5wk5eucxa5h9cj05xs9gzc...,502,stake1uypunlj8d82kgsykmugeq6f7qy24s53zh2828rjq...,71406
71397,RUMOR,pool1c30lqt59t8sjn5lg04r5wk5eucxa5h9cj05xs9gzc...,502,stake1uxwp2w24zwvsyvcymxh387ygsc23cfccpmmappda...,55371


Total ADA: 2875242527.1381884


### 3. **Reward Calculation**

- Once we have the delegator history, we can calculate their rewards based on the provided formula.

- This endpoint accepts 3 parameters:


    * **main_rate**: This is the rewards conversion rate of ADA to CRASH for main pool stakes

    * **partner_rate**: This is the rewards conversion rate of ADA to CRASH for partner pool stakes
        * **partner_fixed_amount**: This is not passed to the API, but it is used to calculate the partner rate.

    * **delegator_data**: The delegator data that is fetched from the previous endpoint


- This endpoint calculate the base rewards based on the given formula:

    * Main Pool Reward: **1 ADA = main_rate**

    * Partner Pool Rewards: **1 ADA = partner_rate(partner_fixed_amount / total ADA staked)**

- This endpoint returns all the unique stake addresses with their corresponding total rewards.

#### SAMPLE CALCULATION

![formula image](base_reward_sample.png)

- Let 
    - $ r_m $ = Main Rate
    - $ F_p $ = Partner's fixed reward pool
    - $ r_p $ = Partner's rate
    - $ A_p $ = Total ADA staked

- Therefore:

    - $ r_m $ = 0.025

    - $ F_p $ = 400000

    - $ r_p $ = $ F_p $ / $ A_p $

In [31]:
main_rate = 0.025 

partners_fixed_rewards = 400000 

partner_rate = partners_fixed_rewards / total_ada 

In [32]:
#Reques to the api

rewards_params = {
    "MainRate" : main_rate,
    "PartnerRate": partner_rate,
    "DelegatorData": delegator_data
}

url = "http://localhost:5134/calculate" #endpoint
response = requests.post(url, json=rewards_params)

In [40]:
rewards = response.json()

df = pd.DataFrame(rewards).T.reset_index()
    
    

df.columns = ['Stake Address', 'Total ADA Staked', 'Total Rewards(CRASH)', 'Main Pool Stake Counts']
df["Total ADA Staked"] = df["Total ADA Staked"].round(2)
df["Total Rewards(CRASH)"] = df["Total Rewards(CRASH)"].round(4)
df = df[df["Total Rewards(CRASH)"] != 0]
# Display the rewards per unique address
display(df)
    

,Stake Address,Total ADA Staked,Total Rewards(CRASH),Main Pool Stake Counts
0,stake1u8x78suakz2juuwhatcvwvqrh8frsjegcusc0n75...,10573517.70,264337.9426,49.0
1,stake1uxdtlxetm6xqy8tgt949p04y6x2gzq3vx2w3c0wh...,8166142.79,200092.8254,49.0
2,stake1u9ax4r8t45w355fchdxv0pwaj3cq874lzw6vw5fz...,5269148.33,131728.7081,49.0
3,stake1uya822qae2zg62wmffheq02xenrzvv2yz34m3gcq...,4271304.65,106782.6163,49.0
4,stake1uxhjmz3lwc4tz99tf0y94xe3yluwrkes0lzjwp6w...,4181001.68,104525.0420,49.0
...,...,...,...,...
2153,stake1uy4m589gev0erwuar4n30lr3zrna7qhdppcy4qyz...,4.61,0.0006,0.0
2154,stake1u9cypl2jw6gefp8w6582th9znuy8srexvdnjhmr0...,4.25,0.0006,0.0
2155,stake178jpl03c69shvgh4cgxgjh38egk2zjgqleus0xlz...,3.22,0.0004,0.0
2156,stake1u8hensdknctq8w3z3r6kv29svkf03vr6m87h9tj7...,2.69,0.0004,0.0


### **4. Fetching Asset Holdings(NFTs)**

- This endpoint accepts two parametes:

    - **policy_id**: The policy ID of the NFTs to be fetched.

    - **asset_names**: A list of all the asset names for the 1/1 NFTs.

- This still uses KOIOS API under the hood

- This endpoint returns the following:

    - **one_of_one_asset_holdings**: A list of all asset holdings based on the policy ID and the provided list of asset names.
    
    - **bomber_asset_holdings**: A list of all asset holdings based on the policy ID, specifically for the bomber NFTs.

In [41]:
policy_id = "848838af0c3ab2e3027d420e320c90eb217f25b8b097efb4378e90f5" # Bomber Policy ID

# 1/1 NFTs
asset_names = [
    "426f6d626572732023313137",
    "426f6d62657273202331353130",
    "426f6d62657273202331353034",
    "426f6d62657273202331353033",
    "426f6d62657273202331353038",
    "426f6d626572732023373330",
    "426f6d62657273202331353039",
    "426f6d62657273202331353035",
    "426f6d62657273202331353037",
    "426f6d62657273202331353036",
]

#Request to the API
fetch_asset_params = {
    "PolicyId": policy_id,
    "AssetNames": asset_names
}

url = "http://localhost:5134/fetch-assets" #endpoint
response = requests.post(url, json=fetch_asset_params)

In [ ]:

holdings = response.json()
 
one_of_one_asset_holdings = pd.DataFrame(holdings['oneOfOneAssetHoldings'])
bomber_asset_holdings = pd.DataFrame(holdings['boomerAssetHoldings'])
    
print("1/1 NFT Holdings")
display(one_of_one_asset_holdings)
    
print("Bomber Asset Holdings")
display(bomber_asset_holdings)


1/1 NFT Holdings


,payment_address,stake_address,quantity
0,addr1q97xnj6xuhs4jr5k35jvzc36a9lptruu2tv22y28x...,stake1u8d4jhzn5n9v3cqxxdlayz4z3l2qhly2h2t28wef...,1
1,addr1zxgx3far7qygq0k6epa0zcvcvrevmn0ypsnfsue94...,stake1uxqh9rn76n8nynsnyvf4ulndjv0srcc8jtvumut3...,1
2,addr1q88tjxyuch30cwug44t7cy2unw4mrafestj8ccem6...,stake1u9ckyaqntc50za9t2n5ynyf6cvwmtf474fmlt4kk...,1
3,addr1q9rf4frtulnsatwmqxlckxa2e6qx9kk8xhalc4u7t...,stake1uxs5p4uqjn8tvjtrp4ja9kfhmpqsn2kss058ktnk...,1
4,addr1q88tjxyuch30cwug44t7cy2unw4mrafestj8ccem6...,stake1u9ckyaqntc50za9t2n5ynyf6cvwmtf474fmlt4kk...,1
5,addr1q9rf4frtulnsatwmqxlckxa2e6qx9kk8xhalc4u7t...,stake1uxs5p4uqjn8tvjtrp4ja9kfhmpqsn2kss058ktnk...,1
6,addr1qy8w59meuas2uvepvvp5j4f9rkajmfpuxzr4cr5jm...,stake1u9t5k6vxfnl4c75rx5k5nkfn8qatpv6sjthdugts...,1
7,addr1qy8w59meuas2uvepvvp5j4f9rkajmfpuxzr4cr5jm...,stake1u9t5k6vxfnl4c75rx5k5nkfn8qatpv6sjthdugts...,1
8,addr1q9rf4frtulnsatwmqxlckxa2e6qx9kk8xhalc4u7t...,stake1uxs5p4uqjn8tvjtrp4ja9kfhmpqsn2kss058ktnk...,1
9,addr1qyd80eerr744acpg4xj25pt0y57yl5x9xsfypkwen...,stake1uxam89e5ql8sam8px3qdqdhvyzj8m5lnusrzkkjy...,1


Bomber Asset Holdings


,asset_name,payment_address,stake_address,quantity
0,426f6d62657273202331,addr1q9ck5xd77zr43r0k2fgrsqw63ad77qzl7t50u434v...,stake1uyav3n07w4u6c746z90h4gs2gvy2zxyhvqzhdd74...,1
1,426f6d6265727320233130,addr1qygf8uqefklv83q8574q9jwh6fehprfqvqjhgujwh...,stake1uyxhqwwl7324cfvuwqm2ddq3s4eq5vcmypfy2e3c...,1
2,426f6d626572732023313030,addr1q8f37mn5lnlzhpgjyv9dry79rltqy249g6skgupm0...,stake1uxr4hmmg0rjq2w5s5psku0yemx8ef7sdk3kjc5r6...,1
3,426f6d62657273202331303030,addr1q9kjrghhtgy22vx5hzg0vd2f08lrpknydl7jd6dcr...,stake1u9lkj89wtzhrvf0qwgyedxlgzx08j7qszm6wu3pp...,1
4,426f6d62657273202331303031,addr1q9rf4frtulnsatwmqxlckxa2e6qx9kk8xhalc4u7t...,stake1uxs5p4uqjn8tvjtrp4ja9kfhmpqsn2kss058ktnk...,1
...,...,...,...,...
995,426f6d626572732023353335,addr1q9wqhvxvmzn28ffvnlw6xjpkgqh5kzwyulypkarjc...,stake1uxwh9dhjdlglqmnptutn8xyg9rpngnaxg66f7qgs...,1
996,426f6d626572732023353336,addr1qx70h9zy8wzd43zvh3sdxgu4z85v3r0tdjqxuzgkr...,stake1uxd6autup0xg70rpmcg0ncjwm42xryvfwmkls6w8...,1
997,426f6d626572732023353337,addr1qyd80eerr744acpg4xj25pt0y57yl5x9xsfypkwen...,stake1uxam89e5ql8sam8px3qdqdhvyzj8m5lnusrzkkjy...,1
998,426f6d626572732023353338,addr1qyf0nljypeqa2sx0p500rzjqk5rm9ejk5nlh3klnc...,stake1u988lj4p9m7ymryj65387e6enflymwr8k6rlef6m...,1


### **5. Bonus Calculations Based on Tier**

- The final step to this system is to calculate the bonuses for each delegator. The bonus depends on the tier the delegator falls under, determined by their asset holding and stake. Delegators who hold certain assets are eligible for tiered bonuses, which are calculated and added on top of their base rewards. history.

- This endpoint calculates the bonus reward and adds it on top of the base rewards. It returns the data for all unique addresses with the updated rewards.

- This endpoint accepts two parameters:

    - **rewards**: The data for all unique addresses with their corresponding rewards, obtained from the previous endpoint that calculates the rewards.

    - **holdings**: The data for all asset holdings, including both 1/1 NFTs and bomber holdings, obtained from the previous endpoint that fetches the holdings.

- Here are the tiers:

![tiers image](bonus_tier.png)

In [43]:
# Request to the API

bonus_calculation_params = {
        "Rewards": rewards,
        "AssetHoldings": holdings
    }

url = "http://localhost:5134/add-bonus" #endpoint
response = requests.post(url, json=bonus_calculation_params)

In [45]:

data = response.json()
 
final_rewards = pd.DataFrame(data).T.reset_index()
    
final_rewards.columns = ['Stake Address', 'Total ADA Staked', 'Total Rewards', 'Main Pool Stake Counts']
final_rewards["Total ADA Staked"] = final_rewards["Total ADA Staked"].round(2)
final_rewards["Total Rewards"] = final_rewards['Total Rewards'].round(4)
final_rewards = final_rewards[final_rewards["Total Rewards"] != 0]
    
print("Final Rewards")
display(final_rewards)
    

Final Rewards


,Stake Address,Total ADA Staked,Total Rewards,Main Pool Stake Counts
0,stake1u8x78suakz2juuwhatcvwvqrh8frsjegcusc0n75...,10573517.70,317205.5311,49.0
1,stake1uxdtlxetm6xqy8tgt949p04y6x2gzq3vx2w3c0wh...,8166142.79,240111.3905,49.0
2,stake1u9ax4r8t45w355fchdxv0pwaj3cq874lzw6vw5fz...,5269148.33,158074.4498,49.0
3,stake1uya822qae2zg62wmffheq02xenrzvv2yz34m3gcq...,4271304.65,128139.1395,49.0
4,stake1uxhjmz3lwc4tz99tf0y94xe3yluwrkes0lzjwp6w...,4181001.68,125430.0504,49.0
...,...,...,...,...
2153,stake1uy4m589gev0erwuar4n30lr3zrna7qhdppcy4qyz...,4.61,0.0006,0.0
2154,stake1u9cypl2jw6gefp8w6582th9znuy8srexvdnjhmr0...,4.25,0.0006,0.0
2155,stake178jpl03c69shvgh4cgxgjh38egk2zjgqleus0xlz...,3.22,0.0004,0.0
2156,stake1u8hensdknctq8w3z3r6kv29svkf03vr6m87h9tj7...,2.69,0.0004,0.0
